<a href="https://colab.research.google.com/github/Migaalee/machine_learning/blob/main/Assigment2_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Project description
## Objective
The goal of this assignment is to examine a set of bacterial cell images using machine learning techniques, including feature extraction, features selection and clustering, in order to help the biologists organize similar images. 
You will load all images, extract features, examine them and select a subset for clustering with the goal of reaching some conclusion about the best way of grouping these images.

## Dataset 
Images: bacteria; size: 50x50 pixels, with a black background and the segmented region centered in the image.

# Import all packages that will be used

In [2]:
#import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import parallel_coordinates,radviz, scatter_matrix
import seaborn as sns
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
#from sklearn.preprocessing import StandardScaler -penso que nao devemos fazer standardizacao da data pq muda a sua forma

from sklearn.feature_selection import f_classif #for the anova
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import silhouette_score

#from metrics import conf,accuracy,precision,recall,F1

#Add documents below
#from tp2_aux import report_clusters,images_as_matrix

#Only for google collabs
from google.colab import files #this is to load files from your local disk

# Upload data

Auxiliary document upload

In [3]:
TP2_aux = files.upload() 

Saving tp2_aux.py to tp2_aux.py


Data set

In [4]:
data = files.upload()

Saving images.zip to images.zip


In [5]:
import zipfile
import io
images = zipfile.ZipFile(io.BytesIO(data['images.zip']), "r")
images.extractall()

# Data preparation

[link text](https://)Use the images_as_matrix function to return a 2D numpy array with one image per row (563 rows) and one pixel per column (50x50=2500 columns) from the images in the images folder.

In [6]:
from tp2_aux import images_as_matrix
X = images_as_matrix(N=563)

In [7]:
type(X)
#do we want to shuffle the data?

numpy.ndarray

#Features Extraction

With each method, extract six features from the data set, for a total of 18 features.

###Principal Component Analysis (PCA)
This is a method of dimensionality reduction that computes new features maximizing the preserved variance of the original data set. It transforms the data into an orthogonal set of coordinates so that the values along each coordinate are not correlated. 

In this case, we should not standardize or normalize the data because doing so will distort its shape.


In [8]:
pca = PCA(n_components = 6) #number of features we want to extract that correspond to the principal components
X_pca = pca.fit_transform(X) #create a new matrix X, X_pca, with the new six features 
print(X_pca) #intrepretation: prints the first 6 PC for each image

[[-1.56676906e+03  2.56972339e+02  1.34894384e+02  4.24834982e-01
  -7.82195317e+01  4.23298365e+01]
 [-1.09679053e+03 -2.59430896e+02  2.81108129e+02  1.40741763e+02
   4.86232581e+02  3.37668598e+02]
 [-1.89944666e+03 -8.80539657e+01  1.88662009e+02  1.82423294e+01
   1.10785378e+01 -3.51755873e+02]
 ...
 [-1.60776203e+03 -3.39039523e+02  4.71903438e+02  2.66074514e+02
   4.26675582e+01  2.43051137e+02]
 [ 2.00021657e+02 -5.67829976e+02  9.50199742e+02  1.25975051e+02
  -7.00377365e+02 -7.88102908e+02]
 [-3.20936427e+02  9.14236158e+02  6.23763286e+02  3.20713619e+02
   5.36389825e+02 -2.89595605e+02]]


In [10]:
#Dataframe with the values of the first 6 PCA for all examples 
df_pca = pd.DataFrame(X_pca, columns = ['pca1','pca2','pca3','pca4','pca5','pca6'])
print(df_pca) #WHY DIFFERENT?

"""
#Scatter plot --> useless here I think
pd.plotting.scatter_matrix(df_pca, alpha=0.2) #with histogram by default but can have diagonal="kde"
"""
"""
#Make a Heatmap 
plt.figure(figsize=(20,20))
g=sns.heatmap(X_pca,annot=True,cmap="RdYlGn")
plt.show()
plt.close()
"""


            pca1        pca2         pca3        pca4        pca5         pca6
0   -1566.769060  256.972339   134.894384    0.424835  -78.219532    42.329836
1   -1096.790534 -259.430896   281.108129  140.741763  486.232581   337.668598
2   -1899.446655  -88.053966   188.662009   18.242329   11.078538  -351.755873
3   -1102.846546 -258.014858  -215.959217 -175.719711 -118.139618   -17.901072
4   -1713.854832 -441.738627   -58.189255  111.578862   86.092638   105.252501
..           ...         ...          ...         ...         ...          ...
558   766.613222 -693.642374  1191.752014   43.697103  195.631369  -181.285817
559 -1446.305757  440.791993   555.953448  349.499211 -388.069853  1224.658748
560 -1607.762033 -339.039523   471.903438  266.074514   42.667558   243.051137
561   200.021657 -567.829976   950.199742  125.975051 -700.377365  -788.102908
562  -320.936427  914.236158   623.763286  320.713619  536.389825  -289.595605

[563 rows x 6 columns]


'\n#Make a Heatmap \nplt.figure(figsize=(20,20))\ng=sns.heatmap(X_pca,annot=True,cmap="RdYlGn")\nplt.show()\nplt.close()\n'

In [ ]:
#Make a parallel coordinates plot
#pd.plotting.parallel_coordinates(df_pca, 'pca1', color='#C7F464')

###t-Distributed Stochastic Neighbor Embedding (t-SNE)

In [11]:
tsne = TSNE(n_components = 6, method = 'exact') #method = ’barnes_hut’ is the default. method = 'exact' runs a slower, but exact, algorithm in O(N^2) time. The exact algorithm should be used when nearest-neighbor errors need to be better than 3%. However, the exact method cannot scale to millions of examples.
X_tsne = tsne.fit_transform(X)
print(X_tsne)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[[-47.047035   -8.745543  -21.825912   56.635498   -7.676711    2.081306 ]
 [ -6.69619    -5.126192  -18.78215    -6.4146504   4.6364913 -11.09376  ]
 [ -6.2425556  -2.3074136 -17.659863   -8.153855  -15.935219   -8.623524 ]
 ...
 [ -3.065125   12.534894   -4.7908587 -12.598296   -8.676818  -17.657434 ]
 [ -2.5961785   4.328006   -5.840299   12.989807   18.854675   -2.1212606]
 [  0.6597941   1.2536995  -1.759909   11.583757    4.188132  -20.617422 ]]


In [12]:
#Dataframe
df_tsne = pd.DataFrame(X_tsne,columns = ['tsne1','tsne2','tsne3','tsne4','tsne5','tsne6'])
print(df_tsne)

"""
#Make a Heatmap 
plt.figure(figsize=(20,20))
g=sns.heatmap(X_tsne,annot=True,cmap="RdYlGn")
plt.show()
plt.close()


#Scatter plot --> usekess here I think
pd.plotting.scatter_matrix(df_tsne, alpha=0.2) #with histogram by default but can have diagonal="kde"
"""

         tsne1      tsne2      tsne3      tsne4      tsne5      tsne6
0   -47.047035  -8.745543 -21.825912  56.635498  -7.676711   2.081306
1    -6.696190  -5.126192 -18.782150  -6.414650   4.636491 -11.093760
2    -6.242556  -2.307414 -17.659863  -8.153855 -15.935219  -8.623524
3     6.889652   0.459273  -2.304125  -7.699670 -16.458874  -6.749015
4    -4.590887   2.379746  -5.919497 -18.655733  -6.434440 -10.566350
..         ...        ...        ...        ...        ...        ...
558  15.663663  -4.553642  -3.728080  17.887886   4.941867  -4.350186
559  -2.835900  21.192659  -1.568086  -6.779308  -3.400138  -6.339554
560  -3.065125  12.534894  -4.790859 -12.598296  -8.676818 -17.657434
561  -2.596179   4.328006  -5.840299  12.989807  18.854675  -2.121261
562   0.659794   1.253700  -1.759909  11.583757   4.188132 -20.617422

[563 rows x 6 columns]


'\n#Make a Heatmap \nplt.figure(figsize=(20,20))\ng=sns.heatmap(X_tsne,annot=True,cmap="RdYlGn")\nplt.show()\nplt.close()\n\n\n#Scatter plot --> usekess here I think\npd.plotting.scatter_matrix(df_tsne, alpha=0.2) #with histogram by default but can have diagonal="kde"\n'

###Isometric mapping with Isomap

In [13]:
isomap = Isomap(n_components = 6, n_neighbors=5) #5 é o numero de neighbours por default
X_isomap = isomap.fit_transform(X)
print(X_isomap)

[[-3.54215436e+03  9.85098659e+02  9.03368859e+02 -2.79365740e+02
   1.52162775e+01  1.30851434e+03]
 [-3.15052216e+03 -5.85940269e+02 -2.79961597e+00  5.42739056e+02
   6.55008573e+02  8.13251967e+02]
 [-4.94386583e+03  8.58561244e+01  1.16885161e+03 -1.50175123e+02
  -4.07711430e+02  1.10782630e+03]
 ...
 [-4.56766978e+03  3.68180542e+01  3.49974142e+02  3.90581761e+02
   4.23216752e+02  1.47842810e+03]
 [ 3.36530503e+03 -2.10677432e+02  2.25028050e+03 -7.09276808e+02
  -5.44573532e+02 -3.13341049e+02]
 [-8.96830004e+02  3.10939235e+03 -4.72223891e+02  1.37822745e+03
   7.82954261e+02  6.34254161e+02]]


In [14]:
#Dataframe
df_isomap = pd.DataFrame(X_isomap,columns = ['im1','im2','im3','im4','im5','im6'])
print(df_isomap)

             im1          im2  ...         im5          im6
0   -3542.154360   985.098659  ...   15.216278  1308.514337
1   -3150.522163  -585.940269  ...  655.008573   813.251967
2   -4943.865828    85.856124  ... -407.711430  1107.826303
3   -2305.977768  -639.373716  ... -606.680737    76.015657
4   -3927.758208  -839.460439  ...  613.825335   994.550631
..           ...          ...  ...         ...          ...
558  2725.343066   720.793744  ...  961.556475  -839.706573
559 -4475.871424  1152.987403  ...  371.217944   588.210993
560 -4567.669782    36.818054  ...  423.216752  1478.428098
561  3365.305026  -210.677432  ... -544.573532  -313.341049
562  -896.830004  3109.392351  ...  782.954261   634.254161

[563 rows x 6 columns]


After extracting features, select the best for clustering.

In [52]:
label = files.upload()

Saving labels.txt to labels (1).txt


In [57]:
from pandas.core.frame import DataFrame
labels = np.loadtxt('labels.txt', delimiter=',')
#print(labels)
y = labels[:,1].reshape(-1,1)
y = pd.DataFrame(y, columns = ['y'])

#y.iloc[105] #test that will concatenate well later

y    2.0
Name: 105, dtype: float64

In [58]:
#Create a dataframe with the 18 features
df_total = pd.concat((df_pca, df_tsne, df_isomap,y), axis=1)
print(df_total) 
#df_total.iloc[105]

            pca1        pca2         pca3  ...         im5          im6    y
0   -1566.769060  256.972339   134.894384  ...   15.216278  1308.514337  0.0
1   -1096.790534 -259.430896   281.108129  ...  655.008573   813.251967  0.0
2   -1899.446655  -88.053966   188.662009  ... -407.711430  1107.826303  0.0
3   -1102.846546 -258.014858  -215.959217  ... -606.680737    76.015657  0.0
4   -1713.854832 -441.738627   -58.189255  ...  613.825335   994.550631  0.0
..           ...         ...          ...  ...         ...          ...  ...
558   766.613222 -693.642374  1191.752014  ...  961.556475  -839.706573  0.0
559 -1446.305757  440.791993   555.953448  ...  371.217944   588.210993  0.0
560 -1607.762033 -339.039523   471.903438  ...  423.216752  1478.428098  0.0
561   200.021657 -567.829976   950.199742  ... -544.573532  -313.341049  0.0
562  -320.936427  914.236158   623.763286  ...  782.954261   634.254161  0.0

[563 rows x 19 columns]


pca1     -331.781583
pca2      142.332323
pca3     -835.238129
pca4      478.968555
pca5      -20.307962
pca6      325.679848
tsne1      -2.751650
tsne2     -23.228785
tsne3      -2.171020
tsne4     -11.172799
tsne5       3.680583
tsne6      -0.054389
im1     -1505.950785
im2      -832.815079
im3     -2202.089660
im4      1124.438813
im5      -425.202603
im6       261.101615
y           2.000000
Name: 105, dtype: float64

In [29]:
#Evaluate the most usefull features with ANOVA
f, prob = f_classif(df_total,y) #o q por no y? documento label!!!
print(f) # check highest value here
print(prob) #check smallest value here


[12.79637841 19.24821336 32.71839856  0.63172056  3.43336322  1.89900063
  5.54091665  1.37656129  0.23095464  2.36625393  1.72701792  4.84783988
 14.04786286 41.31820477  5.37225413  5.7198292   6.81730345  1.3927433 ]
[4.25350548e-08 6.86097996e-12 1.70256514e-19 5.94771310e-01
 1.68207478e-02 1.28636995e-01 9.40965175e-04 2.48982401e-01
 8.74836557e-01 6.99984440e-02 1.60325795e-01 2.44202362e-03
 7.69520113e-09 3.96454630e-24 1.18712751e-03 7.35260521e-04
 1.61480141e-04 2.44046422e-01]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#Evaluate the most usefull features with Select K Best - Lecture 15



#Clustering algorithms
For this assignment, you will parametrize and compare at least two clustering algorithms: DBSCAN and K-Means. 

##DBSCAN 
(you should use a value of 5 neighbouring points as this is the default value in the Scikit-Learn implementation of DBSCAN)

##K-Means

##Examine the performance
In addition, examine the performance of each algorithm (K-Means and DBSCAN) by varying the main parameter of each one (neighbourhood distance ε and number of clusters k; you can leave the other parameters with their default values)

##Select some parameter values
Finally, select some parameter values for closer examination by visually inspecting the clusters generated. For this you can use the report_clusters(ids, labels, report_file) function in the tp2_aux.py module.

Considering all the information gathered at this stage, recommend a procedure for the biologists that will help them process the segmented images, both for cell classification and to help discard segmentation errors.

##Optional exercise (2/20) - Bissecting K-Means hierarchical clustering (lecture 19)

Example of hierarchical divisive clustering. The possibility of stopping at any desired level makes this process less time-consuming, although it is a divisive cluster (usually O(2^n)). 